In [3]:
import os

import pd
import px
from numpy import nan_to_num
from oemof import solph

simulation_token = "vw"
simulations_path = os.path.abspath(
    os.path.join(os.getcwd(), '..', 'storage', 'simulations', simulation_token, "dump")
)
print(simulations_path)

dump_path = os.path.join(simulations_path, simulation_token, "dump", "oemof_es.dump")
# Prüfen Sie, ob die Datei existiert und nicht leer ist
if os.path.exists(os.path.join(simulations_path, "oemof_es.dump")):
    file_size = os.path.getsize(os.path.join(simulations_path, "oemof_es.dump"))
    print(f"Dateigröße: {file_size} Bytes")

es = solph.EnergySystem()
es.restore(
    dpath=os.path.join(simulations_path),
    filename='oemof_es.dump'
)

### Ab hier kopieren

components = []
busses = []
result_data = []

for node in es.nodes:
    if isinstance(node, solph.Bus):
        print(f"Bus: {node}")
        busses.append(node)
    else:
        components.append(node)

# TODO: Dat muss nochmal überdacht werden. Schon gut, aber irgendwie weird.
for bus in busses:
    graph_data = []

    for t, g in solph.views.node(es.results["main"], node=bus)["sequences"].items():
        idx_asset = abs(t[0].index(bus) - 1)
        if idx_asset > 0:
            # time_series = EnTimeSeries(
            #     name=str(t[0][1]),
            #     data=nan_to_num(g.values) * pow(-1, idx_asset)
            # )
            time_series = {str(t[0][1]): nan_to_num(g.values) * pow(-1, idx_asset)}
        else:
            # time_series = EnTimeSeries(
            #     name=str(t[0][0]),
            #     data=nan_to_num(g.values) * pow(-1, idx_asset)
            # )
            time_series = {str(t[0][0]): nan_to_num(g.values) * pow(-1, idx_asset)}

        graph_data.append(time_series)

    # bus_data: EnDataFrame = EnDataFrame(
    #     name=f"{bus}",
    #     index=es.timeindex.to_pydatetime(),
    #     data=graph_data
    # )
    bus_data = {bus: {
        "index": es.timeindex.to_pydatetime(),
        "data": graph_data
    }}

    result_data.append(bus_data)

plot_data = result_data
print(f"Plot Data: {plot_data}")

for bus in plot_data["items"]:
    plotname = bus["name"]
    x = bus["index"]
    y = {}

    for lineplot in bus["data"]:
        y[lineplot["name"]] = lineplot["data"]

    dataframe = pd.DataFrame(y, index=x)
    # print(dataframe)

    fig = px.line(
        dataframe,
        title=plotname,
    )
    fig.show()


result_components = []

for component in components:
    result_component_data = {}
    component_data = solph.views.node(es.results["main"], node=component)

    if "sequences" in component_data:
        # behandle die sequences
        result_component_data["energy"] = component_data["sequences"][:8760].sum()


    if "scalars" in component_data:
        result_component_data["invest"] = component_data["scalars"]

    result_components.append(result_component_data)

print(result_components)
#print(result_data)


/Users/pyrokar/Documents/Coding/ensys-gui-new/storage/simulations/vw/dump
Dateigröße: 1351372 Bytes
Bus: Strom
Bus: Raumwärme
Plot Data: [{"<oemof.solph.buses._bus.Bus: 'Strom'>": {'index': array([datetime.datetime(2035, 1, 1, 0, 0),
       datetime.datetime(2035, 1, 1, 1, 0),
       datetime.datetime(2035, 1, 1, 2, 0), ...,
       datetime.datetime(2035, 12, 31, 22, 0),
       datetime.datetime(2035, 12, 31, 23, 0),
       datetime.datetime(2036, 1, 1, 0, 0)], shape=(8761,), dtype=object), 'data': [{'PVanlage': array([0., 0., 0., ..., 0., 0., 0.], shape=(8761,))}, {'Stromspeicher': array([-0., -0., -0., ..., -0., -0., -0.], shape=(8761,))}, {'Stromverbrauch': array([-2.505e-04, -9.420e-05, -9.780e-05, ..., -6.270e-04, -5.310e-04,
       -0.000e+00], shape=(8761,))}, {'Wärmepumpe': array([-0.        , -0.        , -0.        , ..., -0.00139429,
       -0.00425143, -0.        ], shape=(8761,))}, {'Stromnetz': array([2.50500000e-04, 9.42000000e-05, 9.78000000e-05, ...,
       2.02128571e

TypeError: list indices must be integers or slices, not str

In [30]:
import pandas as pd
Grundlast = pd.read_csv('~/Nextcloud/Documents/Energysystemprofiles/Grundlast.csv')
Grundlast

,1
0,1
1,1
2,1
3,1
4,1
...,...
8754,1
8755,1
8756,1
8757,1
